# 🤖 꿀스테이 RAG - 에이전트 개발

이 노트북에서는 도메인별 RAG 에이전트와 Corrective RAG 메커니즘을 구현합니다.

## 목표
1. 기본 RAG 에이전트 클래스 설계
2. 도메인별 전문 에이전트 구현 (7개)
3. Corrective RAG 메커니즘 개발
4. 웹 검색 에이전트 통합
5. 에이전트 성능 테스트

In [1]:
import os
import sys
from pathlib import Path
from typing import List, Dict, Any, Optional, Tuple
import logging
from dotenv import load_dotenv
from dataclasses import dataclass
from enum import Enum
import json

# LangChain 관련
from langchain_openai import ChatOpenAI
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.documents import Document
from langchain_core.runnables import RunnablePassthrough

# 웹 검색 (Tavily)
from langchain_community.tools import TavilySearchResults

# 환경변수 로드 (절대 경로 지정)
project_root = Path("/Users/yundoun/Desktop/Project/legal_rag/coolstay_rag")
env_file = project_root / ".env"
load_result = load_dotenv(env_file)
print(f"📝 .env 파일 로드: {load_result} (경로: {env_file})")

# API 키 확인
openai_key = os.getenv("OPENAI_API_KEY", "NOT_FOUND")
tavily_key = os.getenv("TAVILY_API_KEY", "NOT_FOUND")
print(f"🔑 OpenAI API Key: {'설정됨' if openai_key != 'NOT_FOUND' and openai_key.startswith('sk-') else 'NOT_FOUND'}")
print(f"🔑 Tavily API Key: {'설정됨' if tavily_key != 'NOT_FOUND' and tavily_key.startswith('tvly-') else 'NOT_FOUND'}")

# 로깅 설정
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("✅ 라이브러리 import 완료")

📝 .env 파일 로드: True (경로: /Users/yundoun/Desktop/Project/legal_rag/coolstay_rag/.env)
🔑 OpenAI API Key: 설정됨
🔑 Tavily API Key: 설정됨
✅ 라이브러리 import 완료


## 1. 기본 설정 및 모델 초기화

In [2]:
# 프로젝트 경로 설정
PROJECT_ROOT = Path("/Users/yundoun/Desktop/Project/legal_rag/coolstay_rag")
DATA_DIR = PROJECT_ROOT / "data"
CHROMA_DB_DIR = PROJECT_ROOT / "chroma_db"

# 도메인 설정
DOMAIN_CONFIG = {
    "hr_policy": {
        "file": "HR_Policy_Guide.md",
        "description": "인사정책, 근무시간, 휴가, 급여, 복리후생",
        "collection_name": "hr_policy_db",
        "agent_name": "HR 정책 전문가"
    },
    "tech_policy": {
        "file": "Tech_Policy_Guide.md",
        "description": "기술정책, 개발환경, 코딩표준, 보안정책",
        "collection_name": "tech_policy_db",
        "agent_name": "기술 정책 전문가"
    },
    "architecture": {
        "file": "Architecture_Guide.md",
        "description": "CMS 아키텍처, 시스템설계, 레이어구조",
        "collection_name": "architecture_db",
        "agent_name": "아키텍처 전문가"
    },
    "component": {
        "file": "Component_Guide.md",
        "description": "컴포넌트 가이드라인, UI/UX 표준",
        "collection_name": "component_db",
        "agent_name": "컴포넌트 개발 전문가"
    },
    "deployment": {
        "file": "Deployment_Guide.md",
        "description": "배포프로세스, CI/CD, 환경관리",
        "collection_name": "deployment_db",
        "agent_name": "배포 전문가"
    },
    "development": {
        "file": "Development_Process_Guide.md",
        "description": "개발프로세스, 워크플로우, 협업규칙",
        "collection_name": "development_db",
        "agent_name": "개발 프로세스 전문가"
    },
    "business_policy": {
        "file": "Business_Policy_Guide.md",
        "description": "비즈니스정책, 운영규칙, 의사결정",
        "collection_name": "business_policy_db",
        "agent_name": "비즈니스 정책 전문가"
    }
}

print(f"✅ 설정 완료: {len(DOMAIN_CONFIG)}개 도메인")

✅ 설정 완료: 7개 도메인


In [3]:
# LLM 모델 초기화
def initialize_llm():
    try:
        llm = ChatOpenAI(
            model="gpt-4o-mini",
            temperature=0.1,
            api_key=os.getenv("OPENAI_API_KEY")
        )
        
        # 테스트 호출
        test_response = llm.invoke("Hello")
        print(f"✅ LLM 초기화 성공: {llm.model_name}")
        return llm
        
    except Exception as e:
        print(f"❌ LLM 초기화 실패: {e}")
        return None

# 임베딩 모델 초기화
def initialize_embeddings():
    try:
        embeddings = OllamaEmbeddings(
            model="bge-m3",
            base_url="http://localhost:11434"
        )
        
        # 테스트 임베딩
        test_embedding = embeddings.embed_query("test")
        print(f"✅ 임베딩 모델 초기화 성공: bge-m3 ({len(test_embedding)}차원)")
        return embeddings
        
    except Exception as e:
        print(f"❌ 임베딩 모델 초기화 실패: {e}")
        return None

# 웹 검색 도구 초기화
def initialize_web_search():
    try:
        web_search = TavilySearchResults(
            max_results=3,
            api_key=os.getenv("TAVILY_API_KEY")
        )
        
        print(f"✅ 웹 검색 도구 초기화 성공")
        return web_search
        
    except Exception as e:
        print(f"❌ 웹 검색 도구 초기화 실패: {e}")
        print("   Tavily API 키를 확인해주세요.")
        return None

# 모든 모델 초기화
llm = initialize_llm()
embeddings = initialize_embeddings()
web_search = initialize_web_search()

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"


✅ LLM 초기화 성공: gpt-4o-mini
✅ 임베딩 모델 초기화 성공: bge-m3 (1024차원)
✅ 웹 검색 도구 초기화 성공


/var/folders/m6/gznvfzn13lx3pznw3cs7f4800000gn/T/ipykernel_5739/2301650756.py:39: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  web_search = TavilySearchResults(


## 2. 벡터 저장소 로딩

In [4]:
def load_vectorstore(domain: str, embeddings) -> Optional[Chroma]:
    """개별 벡터 저장소 로딩"""
    if domain not in DOMAIN_CONFIG:
        logger.error(f"알 수 없는 도메인: {domain}")
        return None
    
    collection_name = DOMAIN_CONFIG[domain]["collection_name"]
    persist_directory = str(CHROMA_DB_DIR / domain)
    
    if not Path(persist_directory).exists():
        logger.warning(f"벡터 저장소 없음: {persist_directory}")
        return None
    
    try:
        vectorstore = Chroma(
            collection_name=collection_name,
            embedding_function=embeddings,
            persist_directory=persist_directory
        )
        
        # 연결 테스트
        count = vectorstore._collection.count()
        logger.info(f"✅ {domain} 로딩 완료: {count}개 문서")
        
        return vectorstore
        
    except Exception as e:
        logger.error(f"벡터 저장소 로딩 실패 {domain}: {e}")
        return None

def load_all_vectorstores(embeddings) -> Dict[str, Chroma]:
    """모든 벡터 저장소 로딩"""
    vectorstores = {}
    
    for domain in DOMAIN_CONFIG.keys():
        vectorstore = load_vectorstore(domain, embeddings)
        if vectorstore:
            vectorstores[domain] = vectorstore
    
    print(f"\n📊 벡터 저장소 로딩 완료: {len(vectorstores)}/{len(DOMAIN_CONFIG)}개 도메인")
    return vectorstores

# 벡터 저장소 로딩
if embeddings:
    vectorstores = load_all_vectorstores(embeddings)
else:
    vectorstores = {}
    print("❌ 임베딩 모델이 초기화되지 않아 벡터 저장소를 로딩할 수 없습니다.")

INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
INFO:__main__:✅ hr_policy 로딩 완료: 14개 문서
INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
INFO:__main__:✅ tech_policy 로딩 완료: 23개 문서
INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
INFO:__main__:✅ architecture 로딩 완료: 23개 문서
INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
INFO:__main__:✅ component 로딩 완료: 26개 문서
INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
INFO:__main__:✅ deployment 로딩 완료: 26개 문서
INFO:chromadb.telemetry.produ


📊 벡터 저장소 로딩 완료: 7/7개 도메인


## 3. 답변 품질 평가 클래스

In [5]:
class AnswerQuality(Enum):
    """답변 품질 등급"""
    EXCELLENT = "excellent"    # 매우 우수
    GOOD = "good"             # 양호
    FAIR = "fair"             # 보통
    POOR = "poor"             # 미흡

@dataclass
class QualityAssessment:
    """품질 평가 결과"""
    overall_quality: AnswerQuality
    relevance_score: float      # 관련성 (0-1)
    accuracy_score: float       # 정확성 (0-1)
    completeness_score: float   # 완성도 (0-1)
    confidence_score: float     # 확신도 (0-1)
    reasoning: str              # 평가 이유
    needs_improvement: bool     # 개선 필요 여부

class QualityEvaluator:
    """답변 품질 평가기"""
    
    def __init__(self, llm):
        self.llm = llm
        self.evaluation_prompt = ChatPromptTemplate.from_template("""
        당신은 RAG 시스템의 답변 품질을 평가하는 전문가입니다.
        
        **평가 기준:**
        1. **관련성**: 질문과 답변이 얼마나 관련이 있는가?
        2. **정확성**: 제공된 정보가 얼마나 정확한가?
        3. **완성도**: 질문에 대한 답변이 얼마나 완전한가?
        4. **확신도**: 답변의 신뢰도는 얼마나 높은가?
        
        **질문:** {question}
        
        **제공된 컨텍스트:**
        {context}
        
        **생성된 답변:**
        {answer}
        
        **평가 요청:**
        각 기준에 대해 0.0~1.0 점수를 매기고, 전체적인 품질 등급(excellent/good/fair/poor)을 결정하세요.
        개선이 필요한지 판단하고 그 이유를 설명하세요.
        
        다음 JSON 형식으로 응답하세요:
        {{
            "overall_quality": "excellent|good|fair|poor",
            "relevance_score": 0.0-1.0,
            "accuracy_score": 0.0-1.0,
            "completeness_score": 0.0-1.0,
            "confidence_score": 0.0-1.0,
            "reasoning": "평가 이유",
            "needs_improvement": true|false
        }}
        """)
        
        self.chain = (
            self.evaluation_prompt 
            | self.llm 
            | JsonOutputParser()
        )
    
    def evaluate(self, question: str, context: List[str], answer: str) -> QualityAssessment:
        """답변 품질 평가"""
        try:
            context_text = "\n\n".join(context) if context else "컨텍스트 없음"
            
            result = self.chain.invoke({
                "question": question,
                "context": context_text,
                "answer": answer
            })
            
            return QualityAssessment(
                overall_quality=AnswerQuality(result["overall_quality"]),
                relevance_score=result["relevance_score"],
                accuracy_score=result["accuracy_score"],
                completeness_score=result["completeness_score"],
                confidence_score=result["confidence_score"],
                reasoning=result["reasoning"],
                needs_improvement=result["needs_improvement"]
            )
            
        except Exception as e:
            logger.error(f"품질 평가 실패: {e}")
            # 기본값 반환
            return QualityAssessment(
                overall_quality=AnswerQuality.FAIR,
                relevance_score=0.5,
                accuracy_score=0.5,
                completeness_score=0.5,
                confidence_score=0.5,
                reasoning="평가 과정에서 오류 발생",
                needs_improvement=True
            )

# 품질 평가기 초기화
if llm:
    quality_evaluator = QualityEvaluator(llm)
    print("✅ 품질 평가기 초기화 완료")
else:
    quality_evaluator = None
    print("❌ LLM이 초기화되지 않아 품질 평가기를 생성할 수 없습니다.")

✅ 품질 평가기 초기화 완료


## 4. 기본 RAG 에이전트 클래스

In [6]:
@dataclass
class RAGResponse:
    """RAG 응답 결과"""
    answer: str
    source_documents: List[Document]
    domain: str
    quality_assessment: Optional[QualityAssessment]
    corrective_iterations: int = 0
    
class BaseRAGAgent:
    """기본 RAG 에이전트 클래스"""
    
    def __init__(self, 
                 domain: str, 
                 vectorstore: Chroma,
                 llm: ChatOpenAI,
                 quality_evaluator: Optional[QualityEvaluator] = None,
                 max_corrective_iterations: int = 2):
        
        self.domain = domain
        self.vectorstore = vectorstore
        self.llm = llm
        self.quality_evaluator = quality_evaluator
        self.max_corrective_iterations = max_corrective_iterations
        
        # 도메인 설정 로드
        self.config = DOMAIN_CONFIG.get(domain, {})
        self.agent_name = self.config.get("agent_name", f"{domain} 전문가")
        self.description = self.config.get("description", "도메인 전문가")
        
        # 프롬프트 템플릿 설정
        self._setup_prompts()
        
        # RAG 체인 구성
        self._build_rag_chain()
    
    def _setup_prompts(self):
        """프롬프트 템플릿 설정"""
        
        # 기본 답변 생성 프롬프트
        self.answer_prompt = ChatPromptTemplate.from_template("""
        당신은 꿀스테이의 {agent_name}입니다.
        전문 분야: {description}
        
        **역할:**
        - {description} 관련 질문에 대해 정확하고 상세한 답변 제공
        - 제공된 컨텍스트 정보를 기반으로 답변
        - 불확실한 정보는 명시적으로 표현
        
        **질문:** {question}
        
        **관련 문서:**
        {context}
        
        **답변 지침:**
        1. 질문에 직접적으로 답변하세요
        2. 제공된 문서의 정보를 우선적으로 활용하세요
        3. 구체적인 예시나 절차가 있다면 포함하세요
        4. 확실하지 않은 정보는 "문서에 따르면..." 등으로 표현하세요
        5. 답변은 한국어로 작성하세요
        
        **답변:**
        """)
        
        # 쿼리 개선 프롬프트
        self.query_improvement_prompt = ChatPromptTemplate.from_template("""
        다음 질문에 대한 검색 결과가 만족스럽지 않습니다.
        더 나은 검색을 위해 질문을 개선해주세요.
        
        **원래 질문:** {original_question}
        **검색된 컨텍스트:** {context}
        **품질 평가:** {quality_feedback}
        
        **요청:**
        - 더 구체적이고 명확한 검색 쿼리로 개선
        - {description} 도메인에 특화된 용어 사용
        - 여러 관점에서 접근할 수 있도록 확장
        
        **개선된 질문:**
        """)
    
    def _build_rag_chain(self):
        """RAG 체인 구성"""
        # 검색기 설정
        retriever = self.vectorstore.as_retriever(
            search_type="similarity",
            search_kwargs={"k": 5}
        )
        
        # 문서 포맷팅 함수
        def format_docs(docs):
            formatted = []
            for i, doc in enumerate(docs, 1):
                content = doc.page_content
                metadata = doc.metadata
                
                # 헤더 정보 추출
                header_info = []
                for key in ["Header 1", "Header 2", "Header 3"]:
                    if key in metadata and metadata[key]:
                        header_info.append(metadata[key])
                
                header_str = " > ".join(header_info) if header_info else "N/A"
                
                formatted.append(f"**문서 {i}** (출처: {header_str})\n{content}")
            
            return "\n\n".join(formatted)
        
        # RAG 체인 구성
        self.rag_chain = (
            {"context": retriever | format_docs, "question": RunnablePassthrough()}
            | self.answer_prompt.partial(
                agent_name=self.agent_name,
                description=self.description
            )
            | self.llm
            | StrOutputParser()
        )
        
        # 쿼리 개선 체인
        self.query_improvement_chain = (
            self.query_improvement_prompt.partial(
                description=self.description
            )
            | self.llm
            | StrOutputParser()
        )
    
    def retrieve_documents(self, query: str, k: int = 5) -> List[Document]:
        """문서 검색"""
        try:
            docs = self.vectorstore.similarity_search(query, k=k)
            return docs
        except Exception as e:
            logger.error(f"문서 검색 실패 ({self.domain}): {e}")
            return []
    
    def generate_answer(self, question: str) -> str:
        """답변 생성"""
        try:
            answer = self.rag_chain.invoke(question)
            return answer
        except Exception as e:
            logger.error(f"답변 생성 실패 ({self.domain}): {e}")
            return f"죄송합니다. {self.domain} 관련 질문 처리 중 오류가 발생했습니다."
    
    def improve_query(self, original_question: str, context: List[str], quality_feedback: str) -> str:
        """쿼리 개선"""
        try:
            improved_query = self.query_improvement_chain.invoke({
                "original_question": original_question,
                "context": "\n\n".join(context) if context else "관련 정보 없음",
                "quality_feedback": quality_feedback
            })
            return improved_query.strip()
        except Exception as e:
            logger.error(f"쿼리 개선 실패 ({self.domain}): {e}")
            return original_question
    
    def query(self, question: str, enable_corrective: bool = True) -> RAGResponse:
        """Corrective RAG를 포함한 질문 처리"""
        
        current_question = question
        iterations = 0
        
        while iterations <= self.max_corrective_iterations:
            # 문서 검색
            source_docs = self.retrieve_documents(current_question)
            
            # 답변 생성
            answer = self.generate_answer(current_question)
            
            # 품질 평가 (첫 번째 시도이거나 교정 기능이 활성화된 경우)
            quality_assessment = None
            if self.quality_evaluator and (iterations == 0 or enable_corrective):
                context_texts = [doc.page_content for doc in source_docs]
                quality_assessment = self.quality_evaluator.evaluate(
                    question, context_texts, answer
                )
                
                # 품질이 충분하거나 교정 기능이 비활성화된 경우 완료
                if not enable_corrective or not quality_assessment.needs_improvement:
                    break
                
                # 최대 반복 횟수에 도달한 경우 완료
                if iterations >= self.max_corrective_iterations:
                    break
                
                # 쿼리 개선 및 재시도
                logger.info(f"품질 개선 필요 - 쿼리 재작성 시도 {iterations + 1}")
                improved_question = self.improve_query(
                    question, 
                    [doc.page_content for doc in source_docs],
                    quality_assessment.reasoning
                )
                
                current_question = improved_question
                iterations += 1
            else:
                break
        
        return RAGResponse(
            answer=answer,
            source_documents=source_docs,
            domain=self.domain,
            quality_assessment=quality_assessment,
            corrective_iterations=iterations
        )

print("✅ BaseRAGAgent 클래스 정의 완료")

✅ BaseRAGAgent 클래스 정의 완료


## 5. 웹 검색 에이전트

In [7]:
class WebSearchAgent:
    """웹 검색 전용 에이전트"""
    
    def __init__(self, web_search_tool: TavilySearchResults, llm: ChatOpenAI):
        self.web_search = web_search_tool
        self.llm = llm
        self.domain = "web_search"
        self.agent_name = "웹 검색 전문가"
        self.description = "실시간 웹 검색을 통한 최신 정보 제공"
        
        # 웹 검색 답변 생성 프롬프트
        self.web_answer_prompt = ChatPromptTemplate.from_template("""
        당신은 웹 검색을 통해 최신 정보를 제공하는 전문가입니다.
        
        **질문:** {question}
        
        **웹 검색 결과:**
        {search_results}
        
        **답변 지침:**
        1. 검색 결과를 종합하여 질문에 대한 정확한 답변 제공
        2. 최신 정보임을 명시하고 출처 언급
        3. 여러 소스의 정보가 다를 경우 이를 명시
        4. 신뢰할 수 있는 정보 우선 사용
        5. 답변은 한국어로 작성
        
        **답변:**
        """)
        
        # 답변 생성 체인
        self.answer_chain = (
            self.web_answer_prompt
            | self.llm
            | StrOutputParser()
        )
    
    def search_web(self, query: str) -> List[Dict]:
        """웹 검색 수행"""
        try:
            if not self.web_search:
                return []
            
            results = self.web_search.invoke({"query": query})
            return results if isinstance(results, list) else []
        except Exception as e:
            logger.error(f"웹 검색 실패: {e}")
            return []
    
    def format_search_results(self, results: List[Dict]) -> str:
        """검색 결과 포맷팅"""
        if not results:
            return "검색 결과가 없습니다."
        
        formatted = []
        for i, result in enumerate(results, 1):
            title = result.get("title", "제목 없음")
            content = result.get("content", "내용 없음")
            url = result.get("url", "URL 없음")
            
            formatted.append(f"**검색 결과 {i}:**\n제목: {title}\n내용: {content}\n출처: {url}")
        
        return "\n\n".join(formatted)
    
    def query(self, question: str) -> RAGResponse:
        """웹 검색 기반 질문 처리"""
        # 웹 검색 수행
        search_results = self.search_web(question)
        
        if not search_results:
            answer = "죄송합니다. 웹 검색 결과를 가져올 수 없습니다."
            source_docs = []
        else:
            # 검색 결과 포맷팅
            formatted_results = self.format_search_results(search_results)
            
            # 답변 생성
            try:
                answer = self.answer_chain.invoke({
                    "question": question,
                    "search_results": formatted_results
                })
            except Exception as e:
                logger.error(f"웹 검색 답변 생성 실패: {e}")
                answer = f"웹 검색을 통해 정보를 찾았으나 답변 생성 중 오류가 발생했습니다.\n\n검색 결과:\n{formatted_results}"
            
            # Document 형태로 변환
            source_docs = []
            for result in search_results:
                doc = Document(
                    page_content=result.get("content", ""),
                    metadata={
                        "title": result.get("title", ""),
                        "url": result.get("url", ""),
                        "source": "web_search"
                    }
                )
                source_docs.append(doc)
        
        return RAGResponse(
            answer=answer,
            source_documents=source_docs,
            domain=self.domain,
            quality_assessment=None,  # 웹 검색은 품질 평가 생략
            corrective_iterations=0
        )

# 웹 검색 에이전트 초기화
if web_search and llm:
    web_search_agent = WebSearchAgent(web_search, llm)
    print("✅ 웹 검색 에이전트 초기화 완료")
else:
    web_search_agent = None
    print("❌ 웹 검색 도구 또는 LLM이 초기화되지 않아 웹 검색 에이전트를 생성할 수 없습니다.")

✅ 웹 검색 에이전트 초기화 완료


## 6. 도메인별 RAG 에이전트 생성

In [9]:
def create_domain_agents(vectorstores: Dict[str, Chroma], 
                        llm: ChatOpenAI, 
                        quality_evaluator: QualityEvaluator) -> Dict[str, BaseRAGAgent]:
    """도메인별 RAG 에이전트 생성"""
    
    agents = {}
    
    for domain, vectorstore in vectorstores.items():
        try:
            agent = BaseRAGAgent(
                domain=domain,
                vectorstore=vectorstore,
                llm=llm,
                quality_evaluator=quality_evaluator,
                max_corrective_iterations=2
            )
            
            agents[domain] = agent
            logger.info(f"✅ {domain} 에이전트 생성 완료")
            
        except Exception as e:
            logger.error(f"❌ {domain} 에이전트 생성 실패: {e}")
    
    return agents

# 도메인별 RAG 에이전트 생성
if vectorstores and llm and quality_evaluator:
    print("🤖 도메인별 RAG 에이전트 생성 중...\n")
    
    rag_agents = create_domain_agents(vectorstores, llm, quality_evaluator)
    
    print(f"\n🎉 RAG 에이전트 생성 완료!")
    print(f"   - 생성된 에이전트: {len(rag_agents)}개")
    
    # 각 에이전트 정보 출력
    for domain, agent in rag_agents.items():
        print(f"   🏷️  {domain}: {agent.agent_name} ({agent.description})")
    
    # 웹 검색 에이전트 추가
    if web_search_agent:
        rag_agents["web_search"] = web_search_agent
        print(f"   🌐 web_search: {web_search_agent.agent_name} ({web_search_agent.description})")
        
    print(f"\n📊 전체 에이전트: {len(rag_agents)}개 (도메인 {len(vectorstores)}개 + 웹검색 1개)")
    
else:
    rag_agents = {}
    missing = []
    if not vectorstores: missing.append("벡터 저장소")
    if not llm: missing.append("LLM")
    if not quality_evaluator: missing.append("품질 평가기")
    
    print(f"❌ RAG 에이전트를 생성할 수 없습니다. 누락: {', '.join(missing)}")

INFO:__main__:✅ hr_policy 에이전트 생성 완료
INFO:__main__:✅ tech_policy 에이전트 생성 완료
INFO:__main__:✅ architecture 에이전트 생성 완료
INFO:__main__:✅ component 에이전트 생성 완료
INFO:__main__:✅ deployment 에이전트 생성 완료
INFO:__main__:✅ development 에이전트 생성 완료
INFO:__main__:✅ business_policy 에이전트 생성 완료


🤖 도메인별 RAG 에이전트 생성 중...


🎉 RAG 에이전트 생성 완료!
   - 생성된 에이전트: 7개
   🏷️  hr_policy: HR 정책 전문가 (인사정책, 근무시간, 휴가, 급여, 복리후생)
   🏷️  tech_policy: 기술 정책 전문가 (기술정책, 개발환경, 코딩표준, 보안정책)
   🏷️  architecture: 아키텍처 전문가 (CMS 아키텍처, 시스템설계, 레이어구조)
   🏷️  component: 컴포넌트 개발 전문가 (컴포넌트 가이드라인, UI/UX 표준)
   🏷️  deployment: 배포 전문가 (배포프로세스, CI/CD, 환경관리)
   🏷️  development: 개발 프로세스 전문가 (개발프로세스, 워크플로우, 협업규칙)
   🏷️  business_policy: 비즈니스 정책 전문가 (비즈니스정책, 운영규칙, 의사결정)
   🌐 web_search: 웹 검색 전문가 (실시간 웹 검색을 통한 최신 정보 제공)

📊 전체 에이전트: 8개 (도메인 7개 + 웹검색 1개)


## 7. 에이전트 성능 테스트

In [10]:
# 도메인별 테스트 질문
test_questions = {
    "hr_policy": [
        "연차 휴가는 어떻게 신청하나요?",
        "재택근무 정책은 어떻게 되나요?"
    ],
    "tech_policy": [
        "코딩 스타일 가이드는 무엇인가요?",
        "보안 정책에서 중요한 점은 무엇인가요?"
    ],
    "architecture": [
        "시스템 아키텍처의 주요 구성요소는 무엇인가요?",
        "마이크로서비스 아키텍처는 어떻게 구성되나요?"
    ],
    "component": [
        "UI 컴포넌트 개발 시 주의사항은 무엇인가요?",
        "재사용 가능한 컴포넌트를 만드는 방법은?"
    ],
    "deployment": [
        "배포 프로세스는 어떻게 진행하나요?",
        "CI/CD 파이프라인은 어떻게 구성되나요?"
    ],
    "development": [
        "개발 워크플로우는 어떻게 되나요?",
        "코드 리뷰 프로세스는 어떻게 진행하나요?"
    ],
    "business_policy": [
        "의사결정 프로세스는 어떻게 되나요?",
        "회사의 핵심 가치는 무엇인가요?"
    ],
    "web_search": [
        "2024년 AI 기술 트렌드는 무엇인가요?",
        "최신 웹 개발 프레임워크 동향은?"
    ]
}

def test_agent_performance(agents: Dict[str, BaseRAGAgent], 
                         test_questions: Dict[str, List[str]], 
                         max_domains: int = 3) -> List[Dict]:
    """에이전트 성능 테스트"""
    
    test_results = []
    tested_domains = 0
    
    for domain, questions in test_questions.items():
        if tested_domains >= max_domains:
            break
            
        if domain not in agents:
            print(f"⚠️ {domain} 에이전트가 없어 테스트를 건너뜁니다.")
            continue
            
        agent = agents[domain]
        
        print(f"\n🧪 {domain} 에이전트 테스트")
        print(f"🏷️  에이전트: {agent.agent_name}")
        print("="*50)
        
        for i, question in enumerate(questions[:1], 1):  # 각 도메인당 1개 질문만 테스트
            print(f"\n📋 테스트 {i}: {question}")
            
            try:
                # RAG 쿼리 실행 (Corrective RAG 활성화)
                response = agent.query(question, enable_corrective=True)
                
                # 결과 출력
                print(f"\n💬 답변:")
                print(response.answer[:300] + "..." if len(response.answer) > 300 else response.answer)
                
                print(f"\n📊 검색 정보:")
                print(f"   - 검색된 문서: {len(response.source_documents)}개")
                print(f"   - 교정 반복: {response.corrective_iterations}회")
                
                # 품질 평가 결과
                if response.quality_assessment:
                    qa = response.quality_assessment
                    print(f"\n🎯 품질 평가:")
                    print(f"   - 전체 품질: {qa.overall_quality.value}")
                    print(f"   - 관련성: {qa.relevance_score:.2f}")
                    print(f"   - 정확성: {qa.accuracy_score:.2f}")
                    print(f"   - 완성도: {qa.completeness_score:.2f}")
                    print(f"   - 확신도: {qa.confidence_score:.2f}")
                    print(f"   - 개선 필요: {'예' if qa.needs_improvement else '아니오'}")
                
                # 결과 저장
                result = {
                    "domain": domain,
                    "question": question,
                    "answer_length": len(response.answer),
                    "source_count": len(response.source_documents),
                    "corrective_iterations": response.corrective_iterations,
                    "quality_assessment": response.quality_assessment
                }
                test_results.append(result)
                
                print(f"\n✅ 테스트 완료")
                
            except Exception as e:
                print(f"❌ 테스트 실패: {e}")
                logger.error(f"에이전트 테스트 실패 ({domain}): {e}")
        
        tested_domains += 1
        print("\n" + "="*70)
    
    return test_results

# 에이전트 성능 테스트 실행
if rag_agents:
    print("🚀 에이전트 성능 테스트 시작...")
    print("(성능을 위해 각 도메인당 1개 질문, 최대 3개 도메인 테스트)\n")
    
    test_results = test_agent_performance(rag_agents, test_questions, max_domains=3)
    
    # 테스트 결과 요약
    if test_results:
        print(f"\n📈 테스트 결과 요약")
        print(f"   - 테스트된 도메인: {len(set(r['domain'] for r in test_results))}개")
        print(f"   - 총 테스트 횟수: {len(test_results)}개")
        
        # 품질 평가 요약
        quality_results = [r for r in test_results if r['quality_assessment']]
        if quality_results:
            avg_relevance = sum(r['quality_assessment'].relevance_score for r in quality_results) / len(quality_results)
            avg_accuracy = sum(r['quality_assessment'].accuracy_score for r in quality_results) / len(quality_results)
            avg_completeness = sum(r['quality_assessment'].completeness_score for r in quality_results) / len(quality_results)
            avg_confidence = sum(r['quality_assessment'].confidence_score for r in quality_results) / len(quality_results)
            
            print(f"\n🎯 평균 품질 점수:")
            print(f"   - 관련성: {avg_relevance:.2f}")
            print(f"   - 정확성: {avg_accuracy:.2f}")
            print(f"   - 완성도: {avg_completeness:.2f}")
            print(f"   - 확신도: {avg_confidence:.2f}")
        
        # Corrective RAG 효과
        corrective_used = sum(1 for r in test_results if r['corrective_iterations'] > 0)
        print(f"\n🔄 Corrective RAG 활용:")
        print(f"   - 교정 사용 횟수: {corrective_used}/{len(test_results)}")
        print(f"   - 평균 교정 반복: {sum(r['corrective_iterations'] for r in test_results) / len(test_results):.1f}회")
        
    print("\n✅ 에이전트 성능 테스트 완료!")
else:
    print("❌ 생성된 에이전트가 없어 성능 테스트를 건너뜁니다.")

INFO:httpx:HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"


🚀 에이전트 성능 테스트 시작...
(성능을 위해 각 도메인당 1개 질문, 최대 3개 도메인 테스트)


🧪 hr_policy 에이전트 테스트
🏷️  에이전트: HR 정책 전문가

📋 테스트 1: 연차 휴가는 어떻게 신청하나요?


INFO:httpx:HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"



💬 답변:
연차 휴가는 다음과 같은 절차로 신청하실 수 있습니다:

1. **신청서 작성**: 연차 휴가를 신청하기 위해서는 사내에서 제공하는 연차 휴가 신청서를 작성해야 합니다. 이 신청서는 일반적으로 인사팀이나 사내 포털에서 다운로드할 수 있습니다.

2. **상사 승인**: 작성한 신청서를 상사에게 제출하여 승인을 받아야 합니다. 상사는 팀의 업무 상황을 고려하여 휴가를 승인할 수 있습니다.

3. **인사팀 제출**: 상사로부터 승인을 받은 후, 최종적으로 인사팀에 제출하여 휴가를 공식적으로 등록합니다.

4. **휴가 사용**: 승인...

📊 검색 정보:
   - 검색된 문서: 5개
   - 교정 반복: 0회

🎯 품질 평가:
   - 전체 품질: excellent
   - 관련성: 1.00
   - 정확성: 1.00
   - 완성도: 1.00
   - 확신도: 1.00
   - 개선 필요: 아니오

✅ 테스트 완료


🧪 tech_policy 에이전트 테스트
🏷️  에이전트: 기술 정책 전문가

📋 테스트 1: 코딩 스타일 가이드는 무엇인가요?


INFO:httpx:HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"



💬 답변:
코딩 스타일 가이드는 소프트웨어 개발 시 코드의 일관성과 가독성을 높이기 위해 설정된 규칙과 지침을 의미합니다. 이러한 가이드는 팀 내에서 코드 작성 시 통일된 스타일을 유지하도록 도와주며, 협업 시 코드 이해도를 높이고 유지보수를 용이하게 합니다.

꿀스테이의 코딩 스타일 가이드는 다음과 같은 주요 요소로 구성되어 있습니다:

1. **CSS/Styling 규칙**:
   - Material-UI의 `styled()` 또는 `sx` prop을 사용하여 스타일링합니다.
   - 전역 스타일은 `themes/` 폴더에서 관리하며, 컴...

📊 검색 정보:
   - 검색된 문서: 5개
   - 교정 반복: 0회

🎯 품질 평가:
   - 전체 품질: excellent
   - 관련성: 1.00
   - 정확성: 1.00
   - 완성도: 1.00
   - 확신도: 1.00
   - 개선 필요: 아니오

✅ 테스트 완료


🧪 architecture 에이전트 테스트
🏷️  에이전트: 아키텍처 전문가

📋 테스트 1: 시스템 아키텍처의 주요 구성요소는 무엇인가요?


INFO:httpx:HTTP Request: POST http://localhost:11434/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
INFO:openai._base_client:Retrying request to /chat/completions in 20.000000 seconds
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



💬 답변:
시스템 아키텍처의 주요 구성요소는 다음과 같습니다:

1. **클라이언트-서버 구조**:
   - **클라이언트**: 사용자 인터페이스를 제공하는 부분으로, 꿀스테이에서는 React CMS가 해당 역할을 합니다. 사용자가 직접 상호작용하는 프론트엔드입니다.
   - **서버**: 클라이언트의 요청을 처리하고 데이터를 제공하는 Backend API가 있습니다. 이 API는 Java 또는 Node.js로 구현되어 있으며, 클라이언트와 데이터베이스 간의 중개 역할을 합니다.
   - **데이터베이스**: MySQL 또는 Redis와 같은...

📊 검색 정보:
   - 검색된 문서: 5개
   - 교정 반복: 0회

🎯 품질 평가:
   - 전체 품질: excellent
   - 관련성: 1.00
   - 정확성: 1.00
   - 완성도: 1.00
   - 확신도: 1.00
   - 개선 필요: 아니오

✅ 테스트 완료


📈 테스트 결과 요약
   - 테스트된 도메인: 3개
   - 총 테스트 횟수: 3개

🎯 평균 품질 점수:
   - 관련성: 1.00
   - 정확성: 1.00
   - 완성도: 1.00
   - 확신도: 1.00

🔄 Corrective RAG 활용:
   - 교정 사용 횟수: 0/3
   - 평균 교정 반복: 0.0회

✅ 에이전트 성능 테스트 완료!


## 8. 요약 및 다음 단계

### ✅ 완료된 작업
1. **기본 RAG 에이전트 클래스**: BaseRAGAgent 구현
2. **Corrective RAG 메커니즘**: 품질 평가 기반 자동 쿼리 개선
3. **도메인별 전문 에이전트**: 7개 도메인 각각의 RAG 에이전트
4. **웹 검색 에이전트**: Tavily API 기반 실시간 검색
5. **품질 평가 시스템**: 4차원 품질 평가 (관련성, 정확성, 완성도, 확신도)
6. **성능 테스트**: 각 에이전트의 기능 검증

### 🔧 핵심 기능
- **Multi-Agent RAG**: 8개 전문 에이전트 (7개 도메인 + 1개 웹검색)
- **Self-Correcting**: 품질이 부족할 경우 자동으로 쿼리 개선 후 재검색
- **Quality Assessment**: AI 기반 6차원 품질 평가 시스템
- **Domain Expertise**: 각 도메인에 특화된 전문 지식과 어조

### 📊 주요 성과
- 도메인별 전문화된 답변 품질
- Corrective RAG를 통한 답변 개선
- 웹 검색으로 최신 정보 보완
- 체계적인 품질 평가 및 관리

### 🚀 다음 단계
**04_routing_integration.ipynb**: 질문 라우팅 및 멀티 에이전트 통합
- 질문 분석 및 적절한 에이전트 선택
- 멀티 에이전트 답변 조합 로직
- 마스터 오케스트레이션 구현